python 3.10

verbose = 0  (출력하지 않음 X)
verbose = 1  (정보를 상세하게 출력함)
verbose = 2  (정보를 함축적으로 출력함)

In [1]:
%pip install gymnasium==0.29.1
%pip install panda-gym==3.0.7
%pip install tensorboard
# %pip install torch==2.0.1~2.2.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#%pip install stable-baselines3==2.2.1
#%pip install sb3-contrib==2.2.1
#%pip install rl-zoo3==2.2.1

In [3]:
#%pip install stable-baselines3==2.1.0
#%pip install sb3-contrib==2.1.0
#%pip install rl-zoo3==2.1.0

In [4]:
import panda_gym
import sys
import gymnasium
sys.modules["gym"] = gymnasium
# from stable_baselines3 import DDPG
from sb3_contrib import TQC
from stable_baselines3.common.logger import configure

# set up logger
tmp_path = "/tmp/sb3_log/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])


env = gymnasium.make("PandaPickAndPlace-v3")

model = TQC(policy="MultiInputPolicy", env=env, verbose=1)
model.set_logger(new_logger)
model.train(10_000)
model.save("ppap")

Logging to /tmp/sb3_log/
Reset Object object_0 Position: [-0.06151263 -0.04055677  0.        ]
Reset Object object_1 Position: [-0.11371249  0.05092675  0.        ]
Reset Target target_0 Position: [0.00546838 0.00264519 0.18307725]
Reset Target target_1 Position: [0.12250267 0.03671964 0.01942461]
Checking success for [-0.06151263 -0.04055677  0.        ] to [0.00546838 0.00264519 0.18307725]: 0.19967509459763275 (Threshold: 0.05)
Checking success for [-0.11371249  0.05092675  0.        ] to [0.12250267 0.03671964 0.01942461]: 0.23743790375592436 (Threshold: 0.05)
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: high <= 0